In [759]:
import numpy as np
import cvxopt as co
from scipy import linalg as sla
from scipy import optimize as opt
co.solvers.options['show_progress'] = False

**Начальные у словия задачи**

In [760]:
#inflation coefficiation official data from rosstat
beta = 0.75

In [761]:
#starting capital
k_0 = 1000000 #tenge

In [762]:
#investment return function
f = lambda x: 3*x+2

In [763]:
#profit function
u = lambda x: 5*x**2+4*x+7

In [764]:
#function for minimization
def U (x):
    return np.dot(u(x), beta ** np.arange(x.shape[0]))

In [765]:
#differencial of U
def dU (x):
    return np.asarray([10*x[0]+4, (10*x[1]+4)*beta, (10*x[2]+4)*beta**2])

In [766]:
#sec differencial for Newton
def ddU (x):
    return np.asarray([[10, 0,  0],
                      [0, 10*beta, 0],
                      [0, 0, 10*beta**2]])

# Обычные методы

### 1) Градиентный спуск

In [767]:
def grad_desc (acc, df, start, alpha):
    curr_point = start
    prev_point = np.asarray([0, 0, 0])
    
    operations = 0
    
    
    while(sla.norm(curr_point - prev_point) > acc):
        prev_point = curr_point
        curr_point = curr_point - alpha*df(curr_point)
        operations += 1
    
    print("Время работы:", operations)
    print("Точка минимума функции:", curr_point)
    return 

In [768]:
grad_desc(0.0001, dU, [8000, 8000, 8000], 0.01)

Время работы: 266
Точка минимума функции: [-0.39999999 -0.39999211 -0.39835917]


(Но если мы угадаем с коэффициентом)

In [769]:
grad_desc(0.0001, dU, [8000, 8000, 8000], 0.1)

Время работы: 23
Точка минимума функции: [-0.4        -0.4        -0.39995578]


### 2) Адаптивный градиентный спуск

In [774]:
def adapt_grad_desc (acc, f, df, start):
    curr_point = start
    prev_point = np.asarray([0, 0, 0])
    operations = 0
    
    while(sla.norm(curr_point - prev_point) > acc):
        prev_point = curr_point
        alpha = ternary(lambda x: f(prev_point[0:3] - x*df(prev_point[0:3])))
        curr_point = curr_point - alpha*df(curr_point)
        operations += 1
    
    print("Время работы:", operations)
    print("Точка минимума функции:", curr_point)
    return 

In [775]:
adapt_grad_desc(0.0001, U, dU, [8000, 8000, 8000])

Время работы: 16
Точка минимума функции: [-0.39999252 -0.4        -0.39998794]


### 3) Метод ньютона

In [780]:
def newton(f, df, ddf, start_point, precision, max_iters):
    curr_point = start_point
    for i in range(max_iters):
        prev_point = np.copy(curr_point)
        curr_point = curr_point - sla.inv(ddf(curr_point)) @ df(curr_point)
        if sla.norm(curr_point - prev_point) < precision:
            break
    print('Bремя работы:', i+1)
    print("Точка минимума функции:", curr_point)
    return

In [781]:
newton(U, dU, ddU, np.array([8000, 8000, 8000]), 0.0001, 10000000)

Bремя работы: 2
Точка минимума функции: [-0.4 -0.4 -0.4]


# Методы внутренней точки

In [782]:
#our matrix
A = np.asarray([[1, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0],
                [0, 0, 0, -1, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 1],
                [-1, 0, 0, 4, -1, 0],
                [0, -1, 0, 0, 4, -1]])

B = np.asarray([0, 0, 0, -1000001, 999999, 0, 0, -2, -2])

In [783]:
#checking if point in set
def check (A, B, point):
    return np.all(A@point <= B)

In [784]:
start = np.asarray([8000, 8000, 8000, 1000, 0, 0])
check(-A, -B, start)

False

## Проекция на множество

In [791]:
def proj(x, A, b):
    P = 2 * co.matrix(np.identity(x.shape[0]))
    q = -2 * co.matrix(x.astype('float'))
    G = co.matrix(A.astype('float'))
    h = co.matrix(b.astype('float'))
    sol = co.solvers.qp(P, q, G, h)
    return np.array(sol['x']).ravel()

In [792]:
proj_start = proj(start, -A, -B)

In [793]:
check(-A, -B, proj_start)

True

## Тернарник

In [794]:
def ternary (f):
    left = -1
    right = 1
    while f(left) < f(0) or f(right) < f(0):
        left *= 2
        right *= 2
    while (right - left) > 0.00001:
        if f((2 * left + right) / 3) > f((left + 2 * right) / 3):
            left = (2 * left + right) / 3
        else:
            right = (left + 2 * right) / 3
    return right

Проверка работы:

In [795]:
ternary(lambda x: (x-3)**2)

3.000002778004109

### 1)Адаптивный градиентный спуск на множестве

In [802]:
def adapt_grad_desc (acc, f, df, start):
    curr_point = np.copy(start)
    prev_point = np.asarray([0, 0, 0, 0, 0, 0])
    operations = 0
    
    while(sla.norm(curr_point - prev_point) > acc):
        prev_point = np.copy(curr_point)
        alpha = ternary(lambda x: f(prev_point[0:3] - x * df(prev_point[0:3])))
        curr_point[0:3] = curr_point[0:3] - alpha*df(curr_point[0:3])
        
        if not check(-A, -B, curr_point):
            curr_point = proj(curr_point, -A, -B)
        
        operations += 1
    
    print("Время работы:", operations)
    print("Точка минимума функции:", curr_point)
    return curr_point

In [833]:
grad = adapt_grad_desc(100, U, dU, proj_start)
check(-A, -B, grad)

Время работы: 4
Точка минимума функции: [2.19471924e+02 2.79430937e+02 3.31996247e+02 1.00000000e+06
 1.99270747e+03 6.82966461e+02]


True

### 2)Метод Ньютона

In [834]:
def dUd (x):
    return np.asarray([10*x[0]+4, (10*x[1]+4)*beta, (10*x[2]+4)*beta**2, 0, 0, 0])
def ddUd (x):
    return np.asarray([[10, 0,  0, 0, 0, 0],
                      [0, 10*beta, 0, 0, 0, 0],
                      [0, 0, 10*beta**2, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0]])

In [835]:
def newton(f, df, ddf, start_point, precision, max_iters):
    curr_point = np.copy(start_point)
    for i in range(max_iters):
        prev_point = np.copy(curr_point)
        
        P = co.matrix(ddf(curr_point))
        q = co.matrix(df(curr_point))
        G = co.matrix(-A.astype('float'))
        h = co.matrix(-B.astype('float') + A @ curr_point)
        sol = co.solvers.qp(P, q, G, h)
        alpha = ternary(lambda x: f(curr_point[:3] - x * df(curr_point)[:3]))
        curr_point[:3] += alpha * np.array(sol['x']).ravel()[:3]
        if sla.norm(curr_point - prev_point) < precision:
            break
    print('Bремя работы:', i + 1)
    print("Точка минимума функции:", curr_point)
    return curr_point

In [836]:
newt = newton(U, dUd, ddUd, proj_start, 100, 100000)
check(-A, -B, newt)

Bремя работы: 24
Точка минимума функции: [4.04712139e+02 3.80654948e+02 4.04880058e+02 9.99999006e+05
 1.95836288e+03 2.60390882e+02]


True

### 3)Метод Ньютона (с проекцией)

In [837]:
def newtonv2(f, df, ddf, start_point, precision, max_iters):
    curr_point = np.copy(start_point)
    for i in range(max_iters):
        prev_point = np.copy(curr_point)
        curr_point[0:3] -= sla.inv(ddf(curr_point)[:3]) @ (df(curr_point)[:3])
        if not check(-A, -B, curr_point):
            curr_point = proj(curr_point, -A, -B)
        if sla.norm(curr_point - prev_point) < precision:
            break
    print('Bремя работы:', i+1)
    print("Точка минимума функции:", curr_point)
    return curr_point

In [838]:
newt2 = newtonv2(U, dU, ddU, proj_start, 100, 10000)
check(-A, -B, newt)

Bремя работы: 3
Точка минимума функции: [2.47815806e+02 2.72609926e+02 2.80374085e+02 1.00000000e+06
 1.98384243e+03 6.17017682e+02]


True

## Значения минимизируемой функции

In [839]:
print('Адаптивный градиентный спуск:', U(grad))
print('Метод Ньютона:', U(newt))
print('Метод Ньютона(проекция):', U(newt2))

Адаптивный градиентный спуск: 2109384381029.2334
Метод Ньютона: 2109380472271.5933
Метод Ньютона(проекция): 2109384184301.1697


## Добавление штрафа в функцию

In [278]:
Lhs(proj_start), Rhs(proj_start)

(array([[10.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ],
        [ 0.   ,  7.5  ,  0.   ,  0.   ,  1.   ,  0.   ],
        [ 0.   ,  0.   ,  5.625,  0.   ,  0.   ,  1.   ],
        [ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ]]),
 array([[-80004.96610229],
        [-56462.23184539],
        [-45002.79565186],
        [     0.        ],
        [     0.        ],
        [     0.        ]]))

In [279]:
np.linalg.solve(Lhs(proj_start), Rhs(proj_start))

array([[     0.        ],
       [     0.        ],
       [     0.        ],
       [-80004.96610229],
       [-56462.23184539],
       [-45002.79565186]])

In [260]:
Lhs(proj_start)

array([[10.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -1.   ,  0.   ],
       [ 0.   ,  7.5  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -1.   ],
       [ 0.   ,  0.   ,  5.625,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  4.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -1.   ,  4.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -1.   ],
       [-1.   ,  0.   ,  0.   ,  4.   , -1.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , -1.   ,  0.   ,  0.   ,  4.   , -1.   ,  0.   ,  0.   ]])

In [253]:
print (ddU(proj_start), A[-2:, :],sep='\n')

[[10.     0.     0.   ]
 [ 0.     7.5    0.   ]
 [ 0.     0.     5.625]]
[[-1  0  0  4 -1  0]
 [ 0 -1  0  0  4 -1]]


In [553]:
proj_start[0:3].shape


(3,)

In [243]:
def ternary (f):
    left = -1
    right = 1
    while(not (f(left) > f(0) and f(right) > f(0))):
        left*=2
        right*=2
        while((right - left) > 0.00001):
            var1 = f(left+(right-left)/3)
            var2 = f(left+2*(right-left)/3)
            if(var1 > var2):
                left = left+(right-left)/3
            else:
                right = left+2*(right-left)/3
        return right

In [244]:
ternary(lambda x: U(proj_start[0:3] - x * dU(proj_start[0:3])))

In [728]:
newtonv2(U, dU, ddU, proj_start, 10, 10000)

Bремя работы: 37
Точка минимума функции: [2.46378338e+02 2.71196452e+02 2.78946377e+02 1.00000001e+06
 2.20451941e+03 1.30617907e+03]


array([2.46378338e+02, 2.71196452e+02, 2.78946377e+02, 1.00000001e+06,
       2.20451941e+03, 1.30617907e+03])

In [738]:
proj_start == proj(proj_start, -A, -B)

array([False, False, False, False, False, False])